# SENG 550 Final Project 

Luka Petrovic UCID: <br>
Logan Boras UCID: <br> 
Kenny Jeon UCID: 30068677

### 1. Set up environment and Spark ###

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# uncomment the following if you do not have spark installed in your project
#!wget --no-check-certificate https://dlcdn.apache.org/spark/spark-3.3.3/spark-3.3.3-bin-hadoop3.tgz
#!tar -xvf spark-3.3.3-bin-hadoop3.tgz
!pip install findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/training/spark-3.3.3-bin-hadoop3"

You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import findspark
import pyspark

findspark.init()
sc = pyspark.SparkContext(appName="Seng550Project")

ConnectionRefusedError: [Errno 111] Connection refused

### 2. Create directory for input data to live ###

Note that the deerfoot.csv file or whatever input data you are using must be within the data folder then proceed to store it on the hdfs

In [ ]:
!mkdir data
!hadoop fs -mkdir -p /inputs
!hadoop fs -put data /inputs

### 3. Grab the specific data we want from the data set ###

In [ ]:
file = '/inputs/data/deerfoot.csv'

deerfootRDD = (sc.textFile(file, 8))
'\n'.join(deerfootRDD.zipWithIndex().map(lambda x: str(x[1]) + ': ' + str(x[0])))

#### 3a. Now we want to get the data only for morning and afternoon rush hour, 8am and 4pm will suffice ####

In [ ]:
def getRushHours(RDD):

    rushHours = RDD.split(',')
    return (rushHours[1],rushHours[6],rushHours[13])

rushHoursRDD = deerfootRDD.map(getRushHours)
print(rushHoursRDD.take(1))

#### 3b. Rush hour generally doesn't happen on weekends, so we need to filter out Saturdays and Sundays ####

In [ ]:
# This function was taken from the Spark Basics Tutorial

def removeWeekends(deerfootRDDRecord):
    if deerfootRDDRecord[0]=="Saturday" or deerfootRDDRecord[0]=="Sunday":
        return False
    else:
        return True

weekdayRushHoursRDD = rushHoursRDD.filter(removeWeekends)
print(weekdayRushHoursRDD.take(5))

### 4. Compute Statistics and Averages

For our analysis, we want to get both the weekly AM and PM averages, as well as the overall average. For a day to be considered "bad", one of the AM or PM commute times should be above the average. (this is subject to change)

#### 4a. Get day counts

In [ ]:
totalCount = weekdayRushHoursRDD.count()
print(totalCount)

# The below might be useful later, uncomment if we need counts of each day in dataset
# perDayRDD = weekdayRushHoursRDD.map(lambda x: (x[0],1))
# perDayCountsRDD = perDayRDD.reduceByKey(lambda x,y:x+y)

# dayCountsList = perDayCountsRDD.collect()
# dayCountsDict = dict(deerfootDayCountsList)

#### 4b. Get rush hour averages

In [3]:
testRdd = sc.parallelize([('Monday', '45', '40'), ('Tuesday', '52', '40'), ('Wednesday', '39', '40')])
totalRushHourSum = sc.accumulator(0) # total average

def sum(item):
    global totalRushHourSum
    totalRushHourSum += int(item[1])
    totalRushHourSum += int(item[2])
    
weekdayRushHoursRDD.foreach(sum)
totalRushHourAverage = totalRushHourSum.value/(totalCount*2)
print(totalRushHourAverage)    



NameError: name 'sc' is not defined